<a href="https://colab.research.google.com/github/sakshamgulati/Kaggle_MentalHealthChallenge/blob/master/HotDog%20or%20Not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 3

This problem's purpose is to build a convolutional neural network to classify images as hot dogs or not-hot dogs. This is the same problem as seen in the HBO TV show "Silicon Valley" (https://www.youtube.com/watch?v=pqTntG1RXSY).  We'll be using the dataset put together by a user on Kaggle (https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog) which contains 498 training images and 500 test images.



# Code

In [0]:
%tensorflow_version 1.x
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,UpSampling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.applications.mobilenet import MobileNet, preprocess_input


Using TensorFlow backend.


## Loading Hotdog-Not-Hotdog Dataset 

In [0]:
# Download files
!wget https://briankeng.com/files/hotdog.tar.gz
!tar -xvzf hotdog.tar.gz

--2020-01-28 21:43:35--  https://briankeng.com/files/hotdog.tar.gz
Resolving briankeng.com (briankeng.com)... 207.38.86.247, 2605:de00:1:1:4a:31:0:b0
Connecting to briankeng.com (briankeng.com)|207.38.86.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46732258 (45M) [application/x-tar]
Saving to: ‘hotdog.tar.gz.1’

hotdog.tar.gz.1     100%[===================>]  44.57M  86.6MB/s    in 0.5s    

2020-01-28 21:43:36 (86.6 MB/s) - ‘hotdog.tar.gz.1’ saved [46732258/46732258]

hotdog/
hotdog/test/
hotdog/test/hot_dog/
hotdog/test/hot_dog/324507.jpg
hotdog/test/hot_dog/800992.jpg
hotdog/test/hot_dog/716049.jpg
hotdog/test/hot_dog/588881.jpg
hotdog/test/hot_dog/570799.jpg
hotdog/test/hot_dog/838604.jpg
hotdog/test/hot_dog/315220.jpg
hotdog/test/hot_dog/612440.jpg
hotdog/test/hot_dog/250715.jpg
hotdog/test/hot_dog/292683.jpg
hotdog/test/hot_dog/291354.jpg
hotdog/test/hot_dog/380963.jpg
hotdog/test/hot_dog/533521.jpg
hotdog/test/hot_dog/558890.jpg
hotdog/test/hot_d

In [0]:
# Re-scaled dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = 'hotdog/train'
test_data_dir = 'hotdog/test'

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [0]:
mobilenet_base = MobileNet(weights='imagenet', include_top=False)
mobilenet_base

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


# Model

In [0]:
def mymodel():
    ''' Improve this model! 
        Simple model from: https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d
    '''
    model = Sequential()
    model.add(UpSampling2D(size=(5,5), input_shape=(32, 32, 3)))
    model.add(mobilenet_base)
    model.add(GlobalAveragePooling2D(input_shape=(32,32,3)))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
  
    model.add(Dense(1, activation='sigmoid'))
    '''model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    '''
    for layer in mobilenet_base.layers:
      layer.trainable = False
    
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], 
                  optimizer='RMSprop')
    
    return model

# Test function
mymodel().summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_1 (UpSampling2 (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenet_1.00_224 (Model)   multiple                  3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
________________________________________

### Loading data on the fly

We load the data directly from the images on disk via these Keras helper functions (`ImageDataGenerator` and `flow_from_directory`). It performs two transformations: 

* Rescaling pixels to be between [0, 1]
* Resizing images to be in `img_width`x`img_height` (150x150)

During training for each batch, the images are read from disk on the fly, loaded into memory and then the transformations are applied.

In [0]:

batch_size = 10
epochs = 10
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Data parameters 
num_train_samples = 498
num_test_samples = 500

# Data generators 
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [0]:
def evaluate_model(runs=5):
    
    scores = [] 
    for i in range(runs):
        print('Executing run %d' % (i+1))
        model = mymodel()
        model.fit_generator(train_generator,
                            callbacks=[],
                            steps_per_epoch=num_train_samples // batch_size,
                            epochs=epochs, verbose=0)
        print(' * Evaluating model on test set')
        scores.append(model.evaluate_generator(test_generator, 
                                               steps=num_test_samples // batch_size,
                                               verbose=0))
        print(' * Test set Loss: %.4f, Accuracy: %.4f' % (scores[-1][0], scores[-1][1]))
        
    accuracies = [score[1] for score in scores]     
    return np.mean(accuracies), np.std(accuracies)
        
mean_accuracy, std_accuracy = evaluate_model(runs=5)

Executing run 1


 * Evaluating model on test set
 * Test set Loss: 0.5843, Accuracy: 0.7740
Executing run 2
 * Evaluating model on test set
 * Test set Loss: 0.4606, Accuracy: 0.8320
Executing run 3
 * Evaluating model on test set
 * Test set Loss: 0.5039, Accuracy: 0.8260
Executing run 4
 * Evaluating model on test set
 * Test set Loss: 0.5393, Accuracy: 0.7980
Executing run 5
 * Evaluating model on test set
 * Test set Loss: 0.6208, Accuracy: 0.7820


In [0]:
 
print('Mean test set accuracy over 5 runs: %.4f +/- %.4f' % (mean_accuracy, std_accuracy))

Mean test set accuracy over 5 runs: 0.8024 +/- 0.0231
